In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/drive/Shareddrives/NLP Project'

Mounted at /content/drive
/content/drive/Shareddrives/NLP Project


In [ ]:
# Import required libraries
!pip install accelerate
!pip install seqeval
!pip install datasets 
!pip install torch
!pip install git+https://github.com/huggingface/transformers
!pip install pytorch-lightning=="1.5.8"

In [ ]:
# import required python libraries
import os
import json
%cd 'notebooks/utils'
import predictions
%cd '/content/drive/Shareddrives/NLP Project'

/content/drive/Shareddrives/NLP Project/notebooks/utils
/content/drive/Shareddrives/NLP Project/notebooks/utils


In [ ]:
%cd 'notebooks/utils'
import importlib
importlib.reload(predictions)
%cd '/content/drive/Shareddrives/NLP Project'

/content/drive/Shareddrives/NLP Project/notebooks/utils
/content/drive/Shareddrives/NLP Project/notebooks/utils


### Additional functions

In [ ]:
def process_attributes(pred_lines, json_lines, model_name):
    doc_dic = {}
    for att_idx, label in enumerate(pred_lines):
        att = json.loads(json_lines[att_idx])
        doc_id = att['note_id']
        doc_dic[doc_id] = doc_dic.get(doc_id, {})
        att_str = f'''{att['rid']}\t{model_name.capitalize()} {att['tid']} {label}\n'''
        doc_dic[doc_id][att['rid']] = att_str
    return doc_dic


In [ ]:
def update_ann_files(context_ann_dir, doc_dic, model_name):
    root_names = [file.replace(".ann", "") for file in 
                  os.listdir(context_ann_dir) if ".ann" in file]

    for root_name in root_names:
        new_lines = []
        ann_file = os.path.join(context_ann_dir, root_name + ".ann")
        ann = open(ann_file, "r")
        lines = ann.readlines()
        for line in lines:
            if line.strip().startswith('A'):
                attr_id, attr_m = line.strip().split('\t')
                attr_type, attr_arg, attr_val = attr_m.split(' ')
                # only update the attributes for this specfic model
                if attr_type == model_name.capitalize():
                    # Append the new text string to be written to the file based on id
                    new_lines.append(doc_dic[root_name][attr_id])
                else:
                    new_lines.append(line)
            else:
              new_lines.append(line)

        # write to new file
        with open(ann_file, 'w') as ann:
            for mention_str in new_lines:
                ann.writelines(mention_str)

### Update ann files for all context dimensions

In [ ]:
import datetime

In [ ]:
# Get most recent model version
input_dict = {'action': None, 
              'actor': None, 
              'certainty': None, 
              'negation': None, 
              'temporality': None}
for attr in input_dict.keys():
    latest_version = predictions.get_best_model(attr)
    input_dict[attr] = latest_version
          
print(input_dict)

{'action': 'version_12-03-2022--16-28-46', 'actor': 'version_12-03-2022--19-32-32', 'certainty': 'version_12-03-2022--16-49-11', 'negation': 'version_12-03-2022--19-03-38', 'temporality': 'version_12-03-2022--19-19-15'}


In [ ]:
# GOLD STANDARD
! cp data/split_data/test/*.ann data/split_data/output/context_predicted_annotations/

In [ ]:
# FULL PIPLELINE
! cp data/split_data/output/event_predicted_annotations_from_ner/*.ann data/split_data/output/context_predicted_annotations_from_ner/

In [ ]:
for suffix in ["", "_from_ner"]:
  
    context_ann_dir = f'data/split_data/output/context_predicted_annotations{suffix}'

    for model_name, ckpt in input_dict.items():

        ckpt_path = f'data/models/{model_name}/{ckpt}/checkpoints'
                      
        input_path = f'data/split_data/input/context_input/{model_name}_test{suffix}.json'

        # load best model
        model = predictions.load_best_model(ckpt_path)

        # Make predictions on test set
        prediction_list = predictions.make_predictions(input_path, model_name, model)

        # process results
        pred_lines, json_lines = predictions.pre_process_results(prediction_list, input_path, model_name)
        doc_dic = process_attributes(pred_lines, json_lines, model_name)

        # save to ann
        update_ann_files(context_ann_dir, doc_dic, model_name)

        print(f"\nTested on {model_name}\n")

03/16/2022 16:00:20 - INFO - sequence_classifier -   Initilazing BaseModel


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

...

Tested on certainty


03/16/2022 16:01:59 - INFO - sequence_classifier -   Initilazing BaseModel
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some w

Tested on negation


03/16/2022 16:02:19 - INFO - sequence_classifier -   Initilazing BaseModel
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some w

Tested on temporality


In [ ]:
# Predicting starting with GOLD STANDARD events
!python3 notebooks/eval_script.py data/split_data/test/ data/split_data/output/context_predicted_annotations/


******************** Evaluation n2c2 2022 Track 1 ********************
************* Contextualized Medication Event Extraction *************

*********************** Medication Extraction ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
                Drug  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000


************************ Event Classification ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
         Disposition  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000
       Nodisposition  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000
        Undetermined  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000
                      ------------------------------------------------
     Overall (micro)  1.0000  1.0000  1.0000    1.0000  1.0000  1.0000
  

In [ ]:
# Predicting starting with PREDICTED events
!python3 notebooks/eval_script.py data/split_data/test/ data/split_data/output/context_predicted_annotations_from_ner/

Files skipped in /content/drive/Shareddrives/NLP Project/data/split_data/test:
100-02.ann, 158-02.ann

******************** Evaluation n2c2 2022 Track 1 ********************
************* Contextualized Medication Event Extraction *************

*********************** Medication Extraction ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
                Drug  0.9519  0.9439  0.9479    0.9760  0.9677  0.9718


************************ Event Classification ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
         Disposition  0.7843  0.8333  0.8081    0.7974  0.8472  0.8215
       Nodisposition  0.8871  0.8836  0.8854    0.9149  0.9112  0.9130
        Undetermined  0.6327  0.5000  0.5586    0.6327  0.5000  0.5586
                      --------------------